<a href="https://colab.research.google.com/github/AlexandrPinevich/Domino_to_1c_export/blob/main/Domino_1C_export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Эта штука должна парсить csv и исходники и рожать из них данные на вход в 1С для ввода начальных остатков. Файлы берутся из гугл диска. Для целей консистентности данных сильно желательно все файлы выгружать из Домина одним днем и одним пакетом

In [ ]:
import pandas as pd            #БЕЗ ЭТОГО РАБОТАТЬ НЕ БУДЕТ!!!
from datetime import datetime  #НАДО ЗАПУСКАТЬ ПЕРЕД ОБРАБОТКОЙ
import pytz                    #ВООБЩЕ ВСЕ ПО ОЧЕРЕДИ НАДО ЗАПУСКАТЬ
!pip install xlsxwriter        #ИНАЧЕ ЭКСЕЛЬ НЕ ПИШЕТСЯ ИЗ-ЗА КОДИРОВКИ

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Пути
output_path = '/content/drive/MyDrive/Colab Notebooks/Output/'
time_now = (datetime.now(pytz.timezone('Europe/Moscow'))\
          .strftime("%Y_%b_%d %H.%M.%S"))
to_xlsx = '.xlsx'
to_csv = '.csv'

## Блок считывания файлов. Папка Input








In [ ]:
# Это читает ИСХОДНИК текущий партии
# предобработанный, но без ручной колонки партий
ishodnic_batch =\
 "/content/drive/MyDrive/Colab Notebooks/Input/!!!!Исходник текущий партии.xlsx"
batch=pd.read_excel(ishodnic_batch, dtype={'ТОВАР': object}
                    , header = 4, skipfooter = 1, keep_default_na=True)
batch.rename(columns={'ТОВАР': 'Партия'}, inplace=True)
batch.rename(columns={'ЕИ': 'ЕИпартии'}, inplace=True)
batch.insert(0,'ТОВАР','')
batch['ТОВАР'] = batch['Партия'].astype('Float64').astype('Int64')
batch.insert(2,'Номенклатура','')
batch['Номенклатура'] = 'ДМ-' + batch['ТОВАР'].apply('{:0>8}'.format).astype(str)
batch.dropna(subset=['ИСХ_КОЛИЧ'], inplace = True)
# Дропает строки без остатка насовсем

In [ ]:
# Это читает ИСХОДНИК текущий уже экселем предобработанный
ishodnic =\
 "/content/drive/MyDrive/Colab Notebooks/Input/!!!!Исходник текущий.xlsx"
df2=pd.read_excel(ishodnic, header = 4, skipfooter = 1, keep_default_na=True)
df2.insert(1,'Номенклатура','')
df2['Номенклатура'] = 'ДМ-' + df2['ТОВАР'].apply('{:0>8}'.format).astype(str)

In [ ]:
# Это читает файл ###Справочники
regs = "/content/drive/MyDrive/Colab Notebooks/Input/###Справочники.xlsx"
unitsDf=pd.read_excel(regs, sheet_name='units', keep_default_na=True\
                          , dtype={'КодЕд': object})
managersDf=pd.read_excel(regs, sheet_name='managers', keep_default_na=True)
warehousesDf=pd.read_excel(regs, sheet_name='warehouses', keep_default_na=True)
batch_statusDf=pd.read_excel(regs, sheet_name='batch_status'\
                             , keep_default_na=True)
suppliersDf=pd.read_excel(regs, sheet_name='suppliers', keep_default_na=True\
                          , dtype={'Поставщик_Номер_в_Домино': object})

In [ ]:
# Это читает BARCODES.CSV сырой, как выпал из отчета Домино
# keep_default_na = 1/0 ПЕРЕКЛЮЧАТЕЛЬ ВЫВОДА NaN
barcodes = "/content/drive/MyDrive/Colab Notebooks/Input/barcodes.csv"
df1=pd.read_csv(barcodes, encoding='Windows-1251',delimiter = ";",header = 0,\
    keep_default_na=True\
    ,names=['ТОВАР','Штрихкод','ТОВАР2','НАИМЕНОВАНИЕ_ДЛИННОЕ',
          'АРТИКУЛ_ДЛИННЫЙ','GrNum','Group','SrgNum','SubGroupeAsIs',
          'supplierCode','supplier','countryCode','country'])
df1.insert(1,'Номенклатура','')
df1['Номенклатура'] = 'ДМ-' + df1['ТОВАР'].apply('{:0>8}'.format).astype(str)
df1['SubGroupe'] = df1['SubGroupeAsIs'].str.upper()

In [ ]:
# Это читает SKU из 1С нужно чтобы только новые карточки можно было добавить
# или страрые удалить
f = "/content/drive/MyDrive/Colab Notebooks/Input/SKU from 1C.xlsx"
from1c=pd.read_excel(f, header = 0, skipfooter = 0, keep_default_na=True)
from1c['Номенклатура'] ='ДМ-' + from1c['Код Домино'].apply('{:0>8}'.format).astype(str)

In [ ]:
# Это читает Категории из 1С 
f = "/content/drive/MyDrive/Colab Notebooks/Input/Категории из 1С.xlsx"
catalog1c=pd.read_excel(f, header = 0, skipfooter = 0, keep_default_na=True)
catalog1c['SubGroupe'] = catalog1c['Подгруппа']

## Блок обработки

In [ ]:
# Фрейм BarNoDup Убирает дубликаты строк из df1 - BARCODES.CSV по коду,
# нужно для корректного джойна с исходником
BarNoDup = df1.copy(deep=True).drop_duplicates(subset=['ТОВАР'])

In [ ]:
# Фрейм BarcodeExport Забираем из таблицы с штрихкодами 2 колонки
# в новый датафрейм BarcodeExport для последующей выгрузки
BarcodeExport = df1[['Номенклатура', 'Штрихкод']].copy(deep=True)
BarcodeExport = BarcodeExport.merge(df2['Номенклатура'], how='inner')
# Оставить только совпавшие c исходником строки,
# Нюансик, строки без штрихкода дропаются тоже how='inner'                                                                   

In [ ]:
# Фрейм skuDf копирует исходник чтобы избежать возможного конфликта
# последовательности загрузки, в нем же добавляем колонки, кот. нужны на импорт
# а не надо здесь ешё наличие штрихкода в домине проверить к этим карточкам?
skuDf = df2.copy(deep=True)
skuDf = skuDf\
    .merge(BarNoDup[['НАИМЕНОВАНИЕ_ДЛИННОЕ','Номенклатура']]\
           , on='Номенклатура', how='left')\
    .merge(BarNoDup[['АРТИКУЛ_ДЛИННЫЙ','Номенклатура']]\
           , on='Номенклатура', how='left')\
    .merge(BarNoDup[['SubGroupe','Номенклатура']]\
           , on='Номенклатура', how='left')\
    .merge(unitsDf[['КодЕд','ЕИ']], on='ЕИ', how='left')\
    .merge(suppliersDf[['Поставщик_Имя','ПОСТАВЩИК']]\
           , on='ПОСТАВЩИК', how='left')\
    .merge(managersDf[['Мен1С','МЕНЕДЖЕР']], on='МЕНЕДЖЕР', how='left')\
    .merge(catalog1c, left_on='SubGroupe', right_on='SubGroupe', how='left')

skuDf.insert(36,'Наименование полное','')
skuDf['Наименование полное'] = skuDf['НАИМЕНОВАНИЕ_ДЛИННОЕ']
skuDf['Направление деятельности'] = 'Основное направление'
skuDf['Использовать партии'] = 'Да'
skuDf['Обязательное заполнение партий'] = 'Да'
skuDf['Нижняя граница остатков'] = skuDf['МИН'].abs()
skuDf['Верхняя граница остатков'] = skuDf['УП'].abs()

In [ ]:
# Фрейм SKUexport Забираем из skuDf нужные колонки и переименовываем 
# для последующей выгрузки в файл
SKUexport =skuDf[['Номенклатура', 'НАИМЕНОВАНИЕ_ДЛИННОЕ',
       'Наименование полное','АРТИКУЛ_ДЛИННЫЙ', 'SubGroupe', 'КодЕд',
       'Поставщик_Имя', 'Тип номенклатуры по умолчанию', 'Вид ставки НДС',
       'Способ списания', 'Счет учета запасов', 'Счет учета затрат',
       'Способ пополнения', 'Вид маркировки', 'Вид маркируемой продукции ИС МП',
       'Обувная продукция', 'Признак предмета расчета',
       'Направление деятельности', 'Использовать партии',
       'Обязательное заполнение партий', 'Нижняя граница остатков',
       'Верхняя граница остатков']].copy(deep=True)
#  'Ценовая группа', # в будущем может пригодиться
SKUexport.rename(columns={'НАИМЕНОВАНИЕ_ДЛИННОЕ' : 'Наименование'
                        ,'АРТИКУЛ_ДЛИННЫЙ' : 'Артикул'
                        ,'SubGroupe' : 'Категория'
                        ,'Тип номенклатуры по умолчанию' : 'ТипНоменклатуры'
                        ,'Способ списания' : 'МетодОценки'
                        ,'КодЕд' : 'Единица измерения'
                        ,'Обязательное заполнение партий' : 'ПроверятьЗаполнениеПартий'
                        ,'Вид маркируемой продукции ИС МП' : 'ВидПродукцииИС'
                        ,'Поставщик_Имя' : 'Поставщик'
                        },inplace=True)
SKUexport = SKUexport.copy(deep=True)

In [ ]:
# Фрейм SKUexportAll (вся номенклатура из исходника)
# Фрейм SKUexportNew (номенклатуры нет в 1с, но есть в исходнике)
# Фрейм SKUexportOld (номенклатуры нет в исходнике, но есть в 1с)
SKUexportAll = SKUexport.copy(deep=True)
SKUexportAll.rename(columns={'Номенклатура': 'Код'}, inplace=True)

SKUexportNew = SKUexport.copy(deep=True)
SKUexportNew = SKUexportNew.merge(from1c[['Код','Номенклатура']], on='Номенклатура', how='left')
SKUexportNew = SKUexportNew[SKUexportNew['Код'].isna()] 
SKUexportNew.drop(columns=['Код'], inplace= True) # все равно пустая
# тут NaN по коду это строки с новыми карточками, вытащим их как раз
SKUexportNew.rename(columns={'Номенклатура': 'Код'}, inplace=True)
 
SKUexportOld = SKUexport.copy(deep=True)
SKUexportOld = SKUexport\
    .merge(from1c[['Код','Номенклатура']], on='Номенклатура', how='right')\
    .merge(BarNoDup[['SubGroupe','Номенклатура']], on='Номенклатура',how='left')
SKUexportOld['ТОВАР'] = SKUexportOld['Код'].str[3:].astype('Int64') #код домино
SKUexportOld = SKUexportOld[SKUexportOld['SubGroupe'].notna()]#оставить тестовый
SKUexportOld = SKUexportOld[SKUexportOld['Наименование'].isna()]
SKUexportOld = SKUexportOld[['Номенклатура','SubGroupe']]
SKUexportOld.rename(columns={'Номенклатура': 'Код'}, inplace=True)
# Все коды из 1с, тут NaN по наименованию это то, что в исходнике не нашлось
# выдает просто список кодов. Не убить тестовые раз. Можно достать из
# баркодов новую группу и заменить, не удалять их 
# merge изначально не работал из-за разного столбца "номенклатура"
# из домино и из 1с, кодировка?

# Переименовать Номенклатуру в Код для 1с пришлось после, костыль

In [ ]:
# Фрейм BarcodeExportNew только штрихкода к новым карточкам
# Проверять наличие уже их в 1с тогда надо бы сразу
BarcodeExportNew = df1[['Номенклатура', 'Штрихкод']].copy(deep=True)
BarcodeExportNew.rename(columns={'Номенклатура': 'Код'}, inplace=True)
BarcodeExportNew = BarcodeExportNew.merge(SKUexportNew['Код']\
                                          , how='inner')
BarcodeExportNew.rename(columns={'Код': 'Номенклатура'}, inplace=True)
# Оставить только совпавшие SKUexportNew строки, новый товар
# Нюансик, строки без штрихкода дропаются тоже how='inner'                                                                   


In [ ]:
# Фрейм PriceExport под штатную загрузку розничной цены и менеджера
PriceExport = skuDf[['Штрихкод', 'ЦЕНА(ПР)','Мен1С']].copy(deep=True)
PriceExport['Штрихкод'] =  PriceExport['Штрихкод'].astype(str)

In [ ]:
# Фрейм партии номенклатуры
batchExport = batch[['Номенклатура', 'Партия',
                     'ПОСТАВЩИК', 'ДОГ']].copy(deep=True)
batchExport = batchExport\
    .merge(batch_statusDf[['СтатусОплаты1С','ДОГ']], on='ДОГ', how='left')\
    .merge(suppliersDf[['Поставщик_Статус_НДС','ПОСТАВЩИК']]\
                        , on='ПОСТАВЩИК', how='left')\
    .merge(suppliersDf[['Поставщик_Имя','ПОСТАВЩИК']]\
                        , on='ПОСТАВЩИК', how='left')
batchExport['Наименование'] = batchExport['Партия'].astype(str)
batchExport['Статус'] = batchExport['СтатусОплаты1С']
batchExport['Владелец партии'] =batchExport['Поставщик_Имя']
batchExport['Налогообложение'] =batchExport['Поставщик_Статус_НДС']

batchExport = batchExport[['Номенклатура','Наименование','Статус', 
                            'Владелец партии','Налогообложение',]]

In [ ]:
# Фрейм bdfExport остатки по партиям
# Комиссия отдельно сделать
# Ошибка если единицы из домина не по списку приходят, case censitive в.т.ч
# Лечить в Домине не дожидаясь перитонита
# в Домине не лечится, возьмем из базовых из исходника
# Допилить проверку ошибок
bdf = batch.copy(deep=True)
bdf['Партия'] = bdf['Партия'].astype(str)

input_rows = bdf.shape[0]
input_qtt = bdf['ИСХ_КОЛИЧ'].sum().round(2)
input_sum = bdf['СебестРуб.3'].sum().round(2)

bdf = bdf\
    .merge(df2[['ЕИ','Номенклатура']]\
           , on='Номенклатура', how='left')\
    .merge(BarNoDup[['Штрихкод','Номенклатура']]\
           , on='Номенклатура', how='left')\
    .merge(BarNoDup[['SubGroupe','Номенклатура']]\
           , on='Номенклатура', how='left')\
    .merge(suppliersDf[['Поставщик_Имя','ПОСТАВЩИК']]\
           , on='ПОСТАВЩИК', how='left')\
    .merge(batch_statusDf[['ВидОперации','ДОГ']]\
           , on='ДОГ', how='left')\
    .merge(unitsDf[['КодЕд','ЕИ']], on='ЕИ', how='left')\
    .merge(unitsDf[['1сКрЕИ','ЕИ']], on='ЕИ', how='left')   

bdf = bdf.sort_values(by=['ВидОперации'])
bdf['Контрагент'] = bdf['Поставщик_Имя']
bdf['Единица измерения'] = bdf['1сКрЕИ']

bdf['Договор'] = 'Основной договор'   
bdf[['1эт остаток', '2эт остаток','скл1 остаток', 'б/н остаток',
           'брак остаток', 'ИСХ_КОЛИЧ','СебестРуб.3']] = bdf[['1эт остаток', 
           '2эт остаток','скл1 остаток', 'б/н остаток',
           'брак остаток', 'ИСХ_КОЛИЧ','СебестРуб.3']].fillna(0)    

bdf['хозу остаток'] =  bdf['ИСХ_КОЛИЧ'] - bdf['1эт остаток']\
                     - bdf['2эт остаток'] - bdf['скл1 остаток']\
                     - bdf['б/н остаток'] - bdf['брак остаток']  
bdf['Себестоимость'] = bdf['СебестРуб.3'] / bdf['ИСХ_КОЛИЧ']

serv_rows = bdf[bdf['ГРУППА'] == '!!УСЛУГИ'].index.value_counts().sum()
serv_qtt = bdf[bdf['ГРУППА'] == '!!УСЛУГИ']['ИСХ_КОЛИЧ'].sum().round(2) 
serv_sum = bdf[bdf['ГРУППА'] == '!!УСЛУГИ']['СебестРуб.3'].sum().round(2)
# У услуг нет остатка

bdf.drop(bdf[(bdf['ГРУППА'] == '!!УСЛУГИ')].index, inplace=True)

# тут мы сцепляем остатки по подразделениям в одну таблицу
batchSum = pd.DataFrame()
storage_domino = warehousesDf['Отдел'].values
storage_1c = warehousesDf['СтруктурнаяЕдиница'].values
i = 0
z = 0
for x in storage_domino:
    # print('step ', i) # отладочное
    # print('x=',x)
    # print(storage_1c[i])
    bdfTmp = bdf[bdf[x] != 0].copy()
    bdfTmp['СтруктурнаяЕдиница'] = storage_1c[i] #Склад 1c
    bdfTmp['Количество'] = bdfTmp[x]
    bdfTmp['Сумма'] = bdfTmp['Количество'] * bdfTmp['Себестоимость']
    z = z + bdfTmp.shape[0]
    # print(bdfTmp.shape[0], 'rows')
    # print(z, 'rowsSum')
    batchSum = pd.concat([batchSum, bdfTmp], ignore_index= True) ######## по одной строке для теста
    i = i + 1

bdfExportAll = batchSum[['Номенклатура', 'Партия', 'Количество',
                      'Единица измерения', 'Сумма',
                      'СтруктурнаяЕдиница', 'Контрагент',  # 'Себестоимость',
                      'ВидОперации', 'Договор'              
                     ]].copy()


output_rows = batchSum.shape[0]
output_qtt = batchSum['Количество'].sum().round(2)
output_sum = batchSum['Сумма'].sum().round(2)


check = pd.DataFrame({'Rows': [input_rows, serv_rows, output_rows],
                     'quantity': [input_qtt, serv_qtt, output_qtt],
                     'sum': [input_sum, serv_sum, output_sum]
                      },index=['Input', 'Service', 'Output'])

check.loc['Check (rows is OK)'] = check.loc['Output'] -\
                              (check.loc['Input'] - check.loc['Service'])

In [ ]:
bdf[(bdf['ГРУППА']  == '!!УСЛУГИ')]

In [ ]:
input_qtt

251904.23

In [ ]:
serv_qtt

-1497.0

In [ ]:
output_qtt.round(2)

253401.23

In [ ]:
# df2.info()

In [ ]:
bdf.info()

In [ ]:
# bdfTmp.info()

In [ ]:
# bdf[bdf['КодЕд'].isna()] 

## Блок проверки констситентности
## Если проверка не прошла, вылезет ошибка, блок записи не запустится
Список возможных глюков на входе:
*   Единицы измерения не из списка, хранятся в партиях, нужно базовые в партии перезаписывать чтобы накатился фикс. А они не записываются, база не дает, я в восхищении!
*   Непечатываемые символы из Домино (Code31) в частности Ищется поиском в экселе =НАЙТИ(СИМВОЛ(33);A1)
   

Придумать как штрихкоды в 1с чистить те, которые поменялись, чтобы всю пачку не грузить и дублей не было



In [ ]:
%whos DataFrame
print('-'*80)
inputLen, outputLen = len(df2), len(SKUexportAll)
if inputLen == outputLen:
    print(f'OK Исходник на входе {inputLen} строк совпадает по длине с итоговым {outputLen} строк')
else:
    raise Exception(f'Количество строк на входе и выходе не совпадает на: {inputLen - outputLen} строк')

noSupplier = SKUexportAll['Поставщик'].isna().sum()   
if noSupplier == 0:
    print(f'OK {noSupplier} строк без поставщика')
else:
    raise Exception(f'{noSupplier} строк без поставщика')

noUnit = SKUexportAll['Единица измерения'].isna().sum() 
if noUnit == 0:
    print(f'OK {noUnit} строк без единицы измерения')
else:
    raise Exception(f'{noUnit} строк без единицы измерения')

print ('На входе df2 исходник текущий строк:',df2.shape[0])
print ('Будет выгружен SKUexportAll строк:',SKUexportAll.shape[0])
print ('Будет выгружен SKUexportNew строк:',SKUexportNew.shape[0])
print ('Будет выгружен SKUexportOld строк:',SKUexportOld.shape[0])
print ('Будет выгружен BarcodeExport строк:',BarcodeExport.shape[0])
print ('Будет выгружен BarcodeExportNew строк:',BarcodeExportNew.shape[0])
print ('Будет выгружен PriceExport строк:',PriceExport.shape[0])
print('-'*80)
print ('Проверка выгрузки партий')
print('-'*80)
print(check)
print('-'*80)
noCheckQtt = check['quantity'][-1]
noCheckSum = check['sum'][-1]
if noCheckQtt == 0 and noCheckSum == 0:
    print(f'Qtt is OK ', noCheckQtt)
    print(f'Sum is OK ', noCheckQtt)
else:
    raise Exception(f'{noCheckQtt},{noCheckQtt} не совпали строки по партиям')

##Блок записи в файлы

In [ ]:
# engine менять обязательно, иначе не сохраняет эксельки
# ВЫГРУЖАЕТ фрейм BarcodeExport в BarcodesAll.xlsx  
file_name = 'BarcodesAll_'
file_name = ''.join((output_path, file_name, time_now, to_xlsx))
BarcodeExport.to_excel(file_name, index=False, header=True\
                       , freeze_panes=(1,1), engine='xlsxwriter') 

In [ ]:
# ВЫГРУЖАЕТ фрейм BarcodeExportNew в BarcodesNew.xlsx  
file_name = 'BarcodesNew_'
file_name = ''.join((output_path, file_name, time_now, to_xlsx))
BarcodeExportNew.to_excel(file_name, index=False, header=True\
                       , freeze_panes=(1,1), engine='xlsxwriter') 

In [ ]:
# ВЫГРУЖАЕТ фрейм PriceExport в Price.xlsx  
file_name = 'Price_'
file_name = ''.join((output_path, file_name, time_now, to_xlsx))
PriceExport.to_excel(file_name, index=False, header=True\
                     , freeze_panes=(1,1), engine='xlsxwriter') 

In [ ]:
# Выгружает Фрейм SKUexportAll в SkuAll.xlsx 
# Справочник всех товаров из исходника для первичного импорта в 1С
file_name = 'SkuAll_'
file_name = ''.join((output_path, file_name, time_now, to_xlsx))
SKUexportAll.to_excel(file_name, index=False, header=True\
                      , freeze_panes=(1,1), engine='xlsxwriter')

In [ ]:
# Выгружает Фрейм SKUexportNew в SkuNew.xlsx 
# Справочник новых товаров из исходника для импорта в 1С
file_name = 'SkuNew_'
file_name = ''.join((output_path, file_name, time_now, to_xlsx))
SKUexportNew.to_excel(file_name, index=False, header=True\
                      , freeze_panes=(1,1), engine='xlsxwriter')

In [ ]:
# Выгружает Фрейм SKUexportOld в SkuOld.xlsx 
# Справочник новых товаров из исходника для импорта в 1С
file_name = 'SkuOld_'
file_name = ''.join((output_path, file_name, time_now, to_xlsx))
SKUexportOld.to_excel(file_name, index=False, header=True\
                      , freeze_panes=(1,1), engine='xlsxwriter')

In [ ]:
# Выгружает Фрейм batchExport в batchExport.xlsx
# Справочник партий, xlsxwriter партии жрет как числа лечится .astype(str)
file_name = 'batchExport_'
file_name = ''.join((output_path, file_name, time_now, to_xlsx))
batchExport.to_excel(file_name, index=False, header=True\
                    , freeze_panes=(1,1), engine='xlsxwriter')

In [ ]:
# Выгружает # Фрейм bdfExportAll в StockAll.xlsx (все строки отладка)
# остатки по партиям
file_name = 'StockAll(все строки отладка)_'
file_name = ''.join((output_path, file_name, time_now, to_xlsx))
bdfExportAll.to_excel(file_name, index=False, header=True\
                    , freeze_panes=(1,1), engine='xlsxwriter')

 Шпаргалка

In [ ]:
#ШПАРГАЛКА

# df
# df.columns
# df.head(5)
# df.tail(5)
# df.info()
# df.dtypes
# df['ТОВАР'] #одна колонка
# df[['ТОВАР','1эт остаток','2эт остаток']] #3 колонки
# df['ТОВАР'].hist #гистограмма
# df.describe()
# df['SubGroupe'].value_counts() # подсчет уникальных значений
# df['SubGroupe'].value_counts(dropna=False) #включая пустые
# q = df['Продано'].quantile(0.25)
# df[df['Продано'] < q]

# df = skuDf[skuDf['Счет учета затрат'].isna()] 
# df
# df['SubGroupe'].value_counts(dropna=False)

# df = skuDf[skuDf['Вид маркируемой продукции ИС МП'].notna()] 
# df
# df['SubGroupe'].value_counts(dropna=False)

# skuDf.groupby('В группе',dropna=False)['РозницаРуб'].agg(['count','sum','mean']).sort_values(by='sum', ascending=False)

# BarcodeExport[:10].to_csv #первые 10
# BarcodeExport['ТОВАР'] = BarcodeExport['ТОВАР'].astype('Int64') 
# batch['ТОВАР'] = batch['Партия'].astype('Float64').astype('Int64') #перевод партии сначала из текста с точкой в флоат, потом в инт
# df[df2['1эт остаток'].notna()]
# df[df["col"]. str.contains (" this string ")] поиск в базе 'unicode_escape'
# BarcodeExport['Номенклатура'] = 'ДМ-' + BarcodeExport['ТОВАР'].apply('{:0>8} '. format).astype (str) #приклеить префикс
# bdf.groupby('ГРУППА',dropna=False)['ИСХ_КОЛИЧ'].agg(['count','sum']).sort_values(by='ГРУППА', ascending=True)
# Таблица не мерджится тк в 1с всосали все подгруппы в верхнем регистре
# df_address['country_lower'] = df_address['Country'].str.lower()
#Save
#file_name = '/content/drive/MyDrive/Colab Notebooks/Output/BarcodeExport_{}.csv'.format(datetime.now(pytz.timezone('Europe/Moscow')).strftime("%Y_%b_%d %H.%M.%S"))
#BarcodeExport.to_csv(file_name, sep=";", index=False, header=True, encoding='Windows-1251') 
# ??pd.DataFrame.to_excel
# find = df2[df2['НАИМЕНОВАНИЕ'].str.contains('| '.join('unicode_escape'))]
# find = skuDf[skuDf['НАИМЕНОВАНИЕ'].str.contains('САМОРЕЗ')]
# ??pd.DataFrame.to_excel
# df1 = df1.replace('unicode_escape',' ', regex=True) # В Теории это лечит непечатные символы, ток непонятно, работает ли. Не работает
# !pip install xlsxwriter #for xlsxwriter as engine
# print ('Я дошёл до конца и выполнился')